In [1]:
# import all dependencies
import time
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
executable_path = {"executable_path":"C:/Users/joxu1/Downloads/chromedriver_win32/chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

In [2]:
# visit the NASA Mars News site and scrape headlines
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)
time.sleep(1)
nasa_html = browser.html
nasa_soup = BeautifulSoup(nasa_html, 'html.parser')



In [3]:
article_list = nasa_soup.find('ul', class_='item_list')
first_item = article_list.find('li', class_='slide')
nasa_headline = first_item.find('div', class_='content_title').text
nasa_teaser = first_item.find('div', class_='article_teaser_body').text
print(nasa_headline)
print(nasa_teaser)

NASA Racks Up Two Emmy Nominations for Mission Coverage
JPL's coverage of the Mars InSight mission is among the efforts that will be up for an award in mid-September.


In [6]:
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)
image_html = browser.html
soup = BeautifulSoup(image_html, "html.parser")

image = soup.find("img", class_="thumb")["src"]
featured_image_url = "https://www.jpl.nasa.gov" + image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23341-640x350.jpg


In [5]:
# visit the mars weather report twitter and scrape the latest tweet
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_weather_url)
mars_weather_html = browser.html
mars_weather_soup = BeautifulSoup(mars_weather_html, 'html.parser')

tweets = mars_weather_soup.find('ol', class_='stream-items')
mars_weather = tweets.find('p', class_="tweet-text").text
print(mars_weather)

InSight sol 222 (2019-07-12) low -99.7ºC (-147.5ºF) high -24.8ºC (-12.6ºF)
winds from the SSE at 4.2 m/s (9.4 mph) gusting to 15.6 m/s (34.8 mph)
pressure at 7.60 hPapic.twitter.com/8Q8lyB6SjM


In [24]:
mars_facts_url = "https://space-facts.com/mars/"
browser.visit(mars_facts_url)
mars_data = pd.read_html(mars_facts_url)
mars_df = mars_data[1]
mars_df.columns = ['Description','Value']
mars_df.set_index('Description', inplace=True)
mars_df




,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [27]:
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html_hemispheres = browser.html
soup = BeautifulSoup(html_hemispheres, "html.parser")
mars_hemisphere = []

products = soup.find("div", class_ = "result-list" )
hemispheres = products.find_all("div", class_="item")

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html_hemispheres = browser.html
    soup=BeautifulSoup(html_hemispheres, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    mars_hemisphere.append({"title": title, "img_url": image_url})

In [28]:
mars_hemisphere

[{'title': 'Cerberus Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]